In [16]:
import pickle as pkl
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import collections
from copy import deepcopy

In [2]:
file = open('./train_comments_subs.pkl', 'rb')
data = pkl.load(file)
data, len(data)

([{'id': '00004320bb',
   'ingredients': [['boiling_water'],
    ['cool_whip', 'french_vanilla_cool_whip'],
    ['seedless_watermelon'],
    ['graham_cracker_crust']],
   'subs': ('seedless_watermelon', 'lime')},
  {'id': '0000631d90',
   'ingredients': [['shredded_coconut'],
    ['lean_ground_beef'],
    ['fresh_garlic_clove', 'fresh_garlic_cloves', 'fresh_garlic'],
    ['salt_and_black_pepper'],
    ['lemon_juice'],
    ['soy_sauce'],
    ['cornstarch'],
    ['pineapple_chunk', 'pineapple_chunks', 'pineapple_topping'],
    ['mandarin_orange',
     'mandarin_oranges',
     'mandarin_oranges_in_light_syrup',
     'satsuma_oranges'],
    ['cashew_nut', 'cashew_nuts']],
   'subs': ('lean_ground_beef', 'chicken_breast')},
  {'id': '0000ed95f8',
   'ingredients': [['butter', 'molly_mcbutter'],
    ['brown_sugar'],
    ['egg', 'eggs', 'duck_egg'],
    ['orange_juice_concentrate'],
    ['orange_rind'],
    ['flour'],
    ['baking_soda', 'baking_ammonia'],
    ['salt', 'vegetable_salt', 'low_

In [3]:
df = pd.read_csv('./dataset/full_dataset.csv')
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [4]:
vocab = []
with open('./test.npy', 'rb') as f:
    vocab = np.load(f, allow_pickle=True)

In [5]:
ingredients = [row[1] for row in vocab[1:-1]]
ingredients

[['1%_fat_buttermilk', 'low_fat_buttermilk'],
 ['1%_fat_cottage_cheese'],
 ['10%_cream', '15%_cream'],
 ['100%_bran'],
 ['10_inch_flour_tortilla',
  '10_inch_flour_tortillas',
  '12_inch_flour_tortilla'],
 ['12_inch_pizza_crust'],
 ['18%_table_cream'],
 ['2%_buttermilk'],
 ['2%_cheddar_cheese'],
 ['2%_evaporated_milk'],
 ['2%_fat_cottage_cheese'],
 ['2%_milk', '2%_low_fat_milk'],
 ['2%_mozzarella_cheese'],
 ['35%_cream'],
 ['6_inch_corn_tortilla', '6_inch_corn_tortillas'],
 ['6_inch_flour_tortilla', '6_inch_flour_tortillas'],
 ['7_up', 'cherry_7_up', '7_up_soda', 'diet_7_up'],
 ['80%_lean_ground_beef'],
 ['85%_lean_ground_beef'],
 ['8_inch_flour_tortilla', '8_inch_flour_tortillas', '7_inch_flour_tortilla'],
 ['9"_pastry_pie_shell', '9"_pastry_pie_shells'],
 ['9"_unbaked_pie_shell', '9"_unbaked_pie_shells'],
 ['90%_lean_ground_beef'],
 ['93%_lean_ground_beef'],
 ['95%_lean_ground_beef'],
 ['96%_lean_ground_beef'],
 ['98%_fat_free_cream_of_chicken_soup'],
 ['98%_fat_free_cream_of_mushroo

In [6]:
nerSet = set() 
for recipe in df['NER']:
  for ingredient in ast.literal_eval(recipe):
    nerSet.add(ingredient)
nerSet

{'Non-dairy',
 'red raspberry preserves',
 'peels of orange',
 'chocolate covered candies',
 'Soy sauce dressing',
 'wooden spoons',
 'human breast milk',
 'multi-bran',
 'rye baguette',
 'canful tequila',
 'beet-carrot',
 'tiramisu liqueur',
 '-o',
 'reserved meat broth',
 'Stove Top Chicken Stuffing',
 'mixed dough',
 'Chocolate Melting Disks',
 "'s natural apple sauce",
 'silk soymilk',
 'Chilli sauce',
 'Fresh white',
 'Truly',
 'Unflavored nonstick cooking spray',
 'freshly ground ground pepper',
 'sprinkling thyme',
 'seedless grape jelly',
 'MIX-INS',
 'turkey fillets',
 'robust molasses',
 'toppings optional',
 'fresh sheeps milk ricotta',
 'basmati white rice',
 'packagedry Jello gelatin',
 'white part only orange rind',
 'ROASTED',
 'while onion',
 'asigo cheese',
 'Alinos',
 'fifth champagne',
 'white allspice',
 'pumpkin pie',
 'white beets',
 'frehsly ground black pepper',
 'well mashed potatoes',
 'chocolate mint layer',
 'canned prune',
 'bite nachos',
 'veggie filling',

In [7]:
len(nerSet)

234059

In [8]:
len(ingredients)

6632

In [9]:
mappedVocab = {}
for entity in ingredients:
  for ingredient in entity:
    if ingredient in nerSet:
      mappedVocab[str(entity)] = ingredient
      break
mappedVocab, len(mappedVocab)

({"['abalone']": 'abalone',
  "['absinthe']": 'absinthe',
  "['achiote']": 'achiote',
  "['acorn', 'acorns']": 'acorn',
  "['advocaat']": 'advocaat',
  "['agar']": 'agar',
  "['agave']": 'agave',
  "['ahi']": 'ahi',
  "['aioli']": 'aioli',
  "['ajinomoto']": 'ajinomoto',
  "['ajwain']": 'ajwain',
  "['alcaparrado']": 'alcaparrado',
  "['alcohol']": 'alcohol',
  "['ale']": 'ale',
  "['alfalfa']": 'alfalfa',
  "['allspice']": 'allspice',
  "['almond', 'almonds']": 'almond',
  "['alum']": 'alum',
  "['amaranth']": 'amaranth',
  "['amaretti']": 'amaretti',
  "['amaretto']": 'amaretto',
  "['ammonia']": 'ammonia',
  "['anchovy', 'anchovies']": 'anchovy',
  "['angelica']": 'angelica',
  "['anise']": 'anise',
  "['anisette']": 'anisette',
  "['apple', 'apples', 'winesap_apples', 'apple_ring']": 'apple',
  "['applejack']": 'applejack',
  "['applesauce']": 'applesauce',
  "['apricot', 'apricots', 'canned_apricots']": 'apricot',
  "['aquavit']": 'aquavit',
  "['armagnac']": 'armagnac',
  "['arro

In [20]:
matchedRecipes = []
# copiedData = deepcopy(data)
for recipe in data:
  temp = True
  for entity in recipe['ingredients']:
    if not str(entity) in mappedVocab:
      temp = False
  if temp:
    matchedRecipes.append((recipe, [mappedVocab[str(entity)] for entity in recipe['ingredients']]))
    # recipe['ingredients'] = [mappedVocab[str(entity)] for entity in recipe['ingredients']]
    # matchedRecipes.append(recipe)
    # for ingredient in entity
    # print(ingredient)
matchedRecipes, len(matchedRecipes)

([({'id': '012cbe55b8',
    'ingredients': [['rice', 'steamed_rice'],
     ['milk', 'breast_milk'],
     ['egg', 'eggs', 'duck_egg'],
     ['sugar'],
     ['vanilla'],
     ['raisin', 'raisins', 'yogurt_covered_raisins'],
     ['nutmeg'],
     ['cinnamon']],
    'subs': ('sugar', 'almond_milk')},
   ['rice',
    'milk',
    'egg',
    'sugar',
    'vanilla',
    'raisin',
    'nutmeg',
    'cinnamon']),
  ({'id': '013c5d2d28',
    'ingredients': [['carrot', 'carrots'],
     ['water', 'hot_water'],
     ['butter', 'molly_mcbutter'],
     ['sugar'],
     ['salt', 'vegetable_salt', 'low_sodium_salt']],
    'subs': ('water', 'chicken_broth')},
   ['carrot', 'water', 'butter', 'sugar', 'salt']),
  ({'id': '038369c801',
    'ingredients': [['flour'],
     ['milk', 'breast_milk'],
     ['shortening', 'liquid_shortening'],
     ['margarine', 'margarine_spread', 'becel_margarine'],
     ['sugar'],
     ['vanilla'],
     ['salt', 'vegetable_salt', 'low_sodium_salt']],
    'subs': ('margarine', '

In [36]:
sameRecipes = []
for recipe1mRecipe, recipe in matchedRecipes:
  recipeSet = set(recipe)
  for nerRecipe in df['NER'][:10000]:
    mappedNerRecipe = ast.literal_eval(nerRecipe)
    if len(recipe) > 2 and (len(mappedNerRecipe) == len(recipeSet)):
      if len(list(set(mappedNerRecipe) - recipeSet)) == 1:
        # print((list(recipeSet - set(mappedNerRecipe)), list(set(mappedNerRecipe) - recipeSet)))
        # print(recipeSet, mappedNerRecipe)
        copiedRecipe1mRecipe = deepcopy(recipe1mRecipe)
        copiedRecipe1mRecipe['subs'] = (list(recipeSet - set(mappedNerRecipe))[0], list(set(mappedNerRecipe) - recipeSet)[0])
        # sameRecipes.append(((copiedRecipe1mRecipe, recipe), mappedNerRecipe))
        sameRecipes.append(copiedRecipe1mRecipe)
sameRecipes, len(sameRecipes)

([{'id': '013c5d2d28',
   'ingredients': [['carrot', 'carrots'],
    ['water', 'hot_water'],
    ['butter', 'molly_mcbutter'],
    ['sugar'],
    ['salt', 'vegetable_salt', 'low_sodium_salt']],
   'subs': ('carrot', 'yeast')},
  {'id': '013c5d2d28',
   'ingredients': [['carrot', 'carrots'],
    ['water', 'hot_water'],
    ['butter', 'molly_mcbutter'],
    ['sugar'],
    ['salt', 'vegetable_salt', 'low_sodium_salt']],
   'subs': ('carrot', 'flour')},
  {'id': '038369c801',
   'ingredients': [['flour'],
    ['milk', 'breast_milk'],
    ['shortening', 'liquid_shortening'],
    ['margarine', 'margarine_spread', 'becel_margarine'],
    ['sugar'],
    ['vanilla'],
    ['salt', 'vegetable_salt', 'low_sodium_salt']],
   'subs': ('margarine', 'butter')},
  {'id': '0a36b05483',
   'ingredients': [['rice', 'steamed_rice'],
    ['salt', 'vegetable_salt', 'low_sodium_salt'],
    ['water', 'hot_water'],
    ['butter', 'molly_mcbutter']],
   'subs': ('rice', 'long grain rice')},
  {'id': '0d01025ef7'

In [38]:
file = open('./nerSubsitutions.pkl', 'wb')

pkl.dump(sameRecipes, file)

In [39]:
file = open('./nerSubsitutions.pkl', 'rb')
sdf = pkl.load(file)
sdf, len(sdf)

([{'id': '013c5d2d28',
   'ingredients': [['carrot', 'carrots'],
    ['water', 'hot_water'],
    ['butter', 'molly_mcbutter'],
    ['sugar'],
    ['salt', 'vegetable_salt', 'low_sodium_salt']],
   'subs': ('carrot', 'yeast')},
  {'id': '013c5d2d28',
   'ingredients': [['carrot', 'carrots'],
    ['water', 'hot_water'],
    ['butter', 'molly_mcbutter'],
    ['sugar'],
    ['salt', 'vegetable_salt', 'low_sodium_salt']],
   'subs': ('carrot', 'flour')},
  {'id': '038369c801',
   'ingredients': [['flour'],
    ['milk', 'breast_milk'],
    ['shortening', 'liquid_shortening'],
    ['margarine', 'margarine_spread', 'becel_margarine'],
    ['sugar'],
    ['vanilla'],
    ['salt', 'vegetable_salt', 'low_sodium_salt']],
   'subs': ('margarine', 'butter')},
  {'id': '0a36b05483',
   'ingredients': [['rice', 'steamed_rice'],
    ['salt', 'vegetable_salt', 'low_sodium_salt'],
    ['water', 'hot_water'],
    ['butter', 'molly_mcbutter']],
   'subs': ('rice', 'long grain rice')},
  {'id': '0d01025ef7'